In [1]:
from simplicial_paths import *

In [ ]:
def hole_exp(n_side=15, seed=1):
    np.random.seed(seed)
    pts1, pts2 = generate_pts(0, n_side), generate_pts(1, n_side)

    S1, S2 = SimplicialComplex(pts1), SimplicialComplex(pts2)
    S = [S1, S2]
    
    hole_locs = [(-0.5, 0.5), (0.5, -0.5)]
    radii = (np.arange(2, 8)) / 20

    # assume all edges have unit weight

    for j, r in enumerate(radii):
        #print(str(j+1) + '/' + str(radii.shape[0]))
        plt.figure(figsize=(8,8))
        for i, s in enumerate(S):
            all_faces = np.arange(s.face_vec.shape[0])
            faces_to_add = {2:set(all_faces)}
            s.add_simplices(faces_to_add)

            s.make_holes(hole_locs, r)

            link_nodes, link_edges = find_hole_edges(s, False)

            x = np.ones(np.sum(s.edge_vec))

            plt.subplot(2, 2, 2 * i + 1)
            plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
            s.plot()

            plt.subplot(2, 2, 2 * i + 2)
            plt.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)
            y1 = s.harm_proj(x)
            y2 = s.harm_proj(link_edges)
            y2_plot = y2[np.linalg.norm(y2, axis=1) > 0]

            plt.scatter(y1[:,0], y1[:,1], color='blue')
            plt.scatter(y2_plot[:,0], y2_plot[:,1], color='red')
        
            plt.tick_params(left = False, right = False , labelleft = False ,
                            labelbottom = False, bottom = False)

        plt.show()
        #plt.savefig('OT_SCs/demo_figs/fig_n' + str(n_side) + '_' + str(j) + '.png')
        #plt.clf()

def demo2(n_side, hole_locs, r):
    np.random.seed(1)
    pts = generate_pts(0, n_side)
    S = SimplicialComplex(pts)
    all_faces = np.arange(S.face_vec.shape[0])
    faces_to_add = {2:set(all_faces)}
    S.add_simplices(faces_to_add)

    S.make_holes(hole_locs, r)
    link_nodes, link_edges = find_hole_edges(S, True)

    x_hat = np.array([not link_edges[i]for i in range(link_edges.shape[0])])
    y1 = S.harm_proj(x_hat)
    y2 = S.harm_proj(link_edges)
    y1_plot = y1[np.linalg.norm(y1, axis=1) > 0]
    y2_plot = y2[np.linalg.norm(y2, axis=1) > 0]

    plt.figure()
    plt.scatter(y1_plot[:,0], y1_plot[:,1], color='blue')
    plt.scatter(y2_plot[:,0], y2_plot[:,1], color='red')

def toy_example():
    pts = np.array([[0,2], [2,1], [0,0], [-2,1], [-2,-3], [0,-2], [2,-3]])
    S = SimplicialComplex(pts)
    all_faces = np.arange(S.face_vec.shape[0])
    faces_to_add = {2:set(all_faces)}
    S.add_simplices(faces_to_add)
    faces_to_remove = set([0])
    edges_to_remove = set([4, 5, 9])
    S.remove_simplices({2:faces_to_remove, 1:edges_to_remove})

    ind = np.where(S.edge_vec == 1)[0]
    edges = [e for i, e in enumerate(S.edges) if i in ind]

    B0, B1 = S.get_incidence_matrices()

    L = B0.T @ B0 + B1 @ B1.T

    print("=*=*" * 15)
    print("B0 incidence matrix")
    print(B0)
    print("=*=*" * 15)
    print("B1 incidence matrix")
    print(B1)
    print("=*=*" * 15)

    c = np.array([-4,-2,4,-2,3,-7,7,3,4,-4])

    p = np.linalg.lstsq(B0.T, c, rcond=None)[0].T
    w = np.linalg.lstsq(B1, c, rcond=None)[0].T

    g = B0.T @ p
    r = B1 @ w

    h = c - g - r

    h = np.round(h, 2)
    g = np.round(g, 2)
    r = np.round(r, 2)

    results = np.vstack([g,r,h,np.round(h+g+r,2)]).T


    print("                    Results")
    print('------------------------------------------------')
    print('edge       ', f"{'grad' : <10}{'curl' : ^10}{'harm' : ^10}{'c' : >5}")
    print('------------------------------------------------')
    for i in range(results.shape[0]):
        print(str(edges[i]) + '     ', f"{results[i,0] : <10}{results[i,1] : ^10}{results[i,2] : ^10}{results[i,3] : >5}")

    print("=*=*" * 15)
    print("L @ h = ", np.round(L @ h, 6))

    plt.figure()
    S.plot()
    plt.show()

In [ ]:
    #toy_example()
    n_side = 20
    hole_locs = [(-0.5, 0.5), (0.5, -0.5)]
    radius = 0.1
    hole_exp(n_side)
    #demo2(n_side, hole_locs, radius)
    #hole_locs = [(-0.25, 0.25), (0.25, -0.25)]
    #demo2(n_side, hole_locs, radius)
    plt.show()
    #hole_exp(n_side=21)
    #hole_exp(n_side=15)
    #hole_exp(n_side=21)